In [1]:
!pip install spacy tabula https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.0 MB/s eta 0:00:00a 0:00:01


In [2]:
# Packages
import os
import re
import pdfplumber
import numpy
import pandas as pd
import spacy
from datetime import datetime

In [3]:
# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Load the provider info CSV for ending keywords
provider_info = pd.read_csv('provider.csv')

# Load the company info CSV for ticker validation and company metadata
company_info = pd.read_csv('company_info.csv')  # Replace with the actual path

company_info = company_info.drop_duplicates(subset='Ticker Symbol')

# Create a dictionary to map ticker symbols to company name and industry
ticker_map = company_info.set_index('Ticker Symbol')[['Company Name', 'Industry']].to_dict(orient='index')

In [4]:
"""# Only relevant to check font type and size for
import pdfplumber

def extract_words_with_formatting(page):

    # Extract words with their bounding boxes
    words = page.extract_words(extra_attrs=["fontname", "size"])

    formatted_words = []
    for word in words:
        formatted_words.append({
            "word": word["text"],
            "font": word.get("fontname", "Unknown"),
            "size": word.get("size", "Unknown"),
            "x0": word["x0"],
            "x1": word["x1"],
            "top": word["top"],
            "bottom": word["bottom"]
        })
    return formatted_words


# Example usage with pdfplumber
pdf_path = "/Users/oskarroeske/Masterthesis/preprocessing/testing/20200605_Needham_CRM_COVID_Hits_CRM_Sales-_Guidance_on_Wrong_Side_of_Growth_-.pdf"

with pdfplumber.open(pdf_path) as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        print(f"Page {page_number}:")
        formatted_words = extract_words_with_formatting(page)
        for word_info in formatted_words:
            print(
                f"Word: '{word_info['word']}', Font: {word_info['font']}, Size: {word_info['size']}, "
                f"Position: ({word_info['x0']}, {word_info['top']} - {word_info['x1']}, {word_info['bottom']})"
            )
"""

'# Only relevant to check font type and size for\nimport pdfplumber\n\ndef extract_words_with_formatting(page):\n\n    # Extract words with their bounding boxes\n    words = page.extract_words(extra_attrs=["fontname", "size"])\n\n    formatted_words = []\n    for word in words:\n        formatted_words.append({\n            "word": word["text"],\n            "font": word.get("fontname", "Unknown"),\n            "size": word.get("size", "Unknown"),\n            "x0": word["x0"],\n            "x1": word["x1"],\n            "top": word["top"],\n            "bottom": word["bottom"]\n        })\n    return formatted_words\n\n\n# Example usage with pdfplumber\npdf_path = "/Users/oskarroeske/Masterthesis/preprocessing/testing/20200605_Needham_CRM_COVID_Hits_CRM_Sales-_Guidance_on_Wrong_Side_of_Growth_-.pdf"\n\nwith pdfplumber.open(pdf_path) as pdf:\n    for page_number, page in enumerate(pdf.pages, start=1):\n        print(f"Page {page_number}:")\n        formatted_words = extract_words_with_

In [5]:
# Test new Structure

patterns = {
    "BGC Partners": {
        "price_patterns": {
            "primary": r"Price Target \(\$\) (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+) \(\w+,",
        },
        "ending_patterns": [r"Disclosures Appendix"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Needham": {
        "price_patterns": {
            "primary": r"Price Target: \$(\d{1,3}(,\d{3})*(\.\d{2})?)",
            "secondary": r"PRICE TARGET: \$(\d{1,3}(,\d{3})*(\.\d{2})?)",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
            "secondary": r"Rating (\w+)",
        },
        "ending_patterns": [r"Analyst Certification"
                            r"ANALYST CERTIFICATION"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Cambria(-Bold)?(-Regular)?", "font_size": 10.0},
            {"font_type": r"[A-Z]+[+]KeplerStd(-Bold)?", "font_size": 9.0},
        ]
    },
    "BTIG": {
        "price_patterns": {
            "primary": r"\$(\d+(\.\d+)?) 12 month target ",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_patterns": [r"Appendix: Analyst Certification and Other Important Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Corbel(,Bold)?(,-Italic)?", "font_size": 9.96},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Wells Fargo": {
        "price_patterns": {
            "primary": r"\/Price Target: \$(\d+(\.\d+)?)",
            "secondary": r"Price Target\/Prior: \$(\d+(\.\d+)?)",
            "tertiary": r"\/\$(\d+(\.\d+)?)"
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+)/\$",
            "secondary": r"Rating (\w+)",
        },
        "ending_patterns": [r"Required Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]WellsFargoSans(-Light)?(,-SemiBold)?", "font_size": 9.00},
            {"font_type": r"[A-Z]+[+]Verdana(-Bold)?", "font_size": 8.04},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 9.01}
        ]
    },
    "Barclays": {
        "price_patterns": {
            "primary": r"Price Target USD (\d+(\.\d+)?)",
            "secondary": r"Price Target: USD (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Stock Rating ([A-Za-z]+)",
        },
        "ending_patterns": [r"ANALYST\(S\) CERTIFICATION\(S\)",
                            r"Analyst\(s\) Certification\(s\)"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Expert Sans (Extra Bold)?(Regular)?(Regular,Bold)?", "font_size": 9.0},
            {"font_type": r"[A-Z]+[+]Expert Sans (Extra Bold)?(Regular)?", "font_size": 8.04},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 9.01},
        ]
    },
    "JP Morgan": {
        "price_patterns": {
            "primary": r"Price Target \([A-Za-z0-9\-]+\): \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b",
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]TimesNewRoman(,Bold)?", "font_size": 9.60},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Brean Capital LLC": {
        "price_patterns": {
            "primary": r"PT: \$ (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+) PT:\$",
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Hilliard Lyons": {
        "price_patterns": {
            "primary": r"Price Target (NA|\$(\d+(\.\d+)?))",
        },
        "rating_patterns": {
            "primary": r"-- NYSE\s+[–\-—]+\s+([A-Za-z]+)\s+[–\-—]+",
            "secondary":r"NYSE\s+[–\-—]+\s+([A-Za-z\- ]+?)(?=\s*[-–—]\d)"
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"Verdana(-Bold)?", "font_size": 9.00},
            {"font_type":r"TimesNewRomanPS(-BoldMT)?", "font_size": 10.98},
            ]
    },
    "Alliance Global Partners": {
        "price_patterns": {
            "primary": r"Price Target (NA|\$(\d+(\.\d+)?))",
        },
        "rating_patterns": {
            "primary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b",
        },
        "ending_patterns": [r"Imporant Research Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]ArialMT(-BoldMT)?", "font_size": 8.00},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Mizuho Securities": {
        "price_patterns": {
            "primary": r"Price Target \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating ([A-Za-z]+)",
        },
        "ending_patterns": [r"IMPORTANT DISCLOSURES"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Gilford Securities Inc": {
        "price_patterns": {
            "primary": r"\, \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rated: ([A-Za-z]+)",
            "secondary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_patterns": r"ANALYST CERTIFICATION",
        "font_patterns": [
            {"font_type": r"ArialMT(-BoldMT)?", "font_size": 10.02},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Deutsche Bank": {
        "price_patterns": {
            "primary": r"Price Target \(USD\) (\d+(\.\d+)?)",
            "secondary": r"Price target (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating ([A-Za-z]+)",
            "secondary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b",
        },
        "ending_patterns": [r"Appendix 1"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]UniversDeutscheBank-Regular", "font_size": 9.0},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Pivotal Research Group": {
        "price_patterns": {
            "primary": r"Target Price: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"RATING: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Appendix: Important Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Helvetica(-Bold)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Arial", "font_size": 8.04},
        ]
    },
    "Spartan Capital Securities LLC": {
        "price_patterns": {
            "primary": r"T \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+)",
        },
        "ending_patterns": [r"Important Disclosures"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Cascend Securities -Historical-": {
        "price_patterns": {
            "primary": r"Price target: \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Disclosures: "],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Calibri(,Bold)?", "font_size": 12.0},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Phillip Securities": {
        "price_patterns": {
            "primary": r"TARGET PRICE USD (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_patterns": [r"Contact Information"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?", "font_size": 10.0},
        ]
    },
    "FinTrust Investment Advisors": {
        "price_patterns": {
            "primary": r"Target Price: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Fintrust Rating: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Important Disclosures:"],
        "font_patterns": [
            {"font_type": r"Arial(-BoldMT)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "IBI Investment House": {
        "price_patterns": {
            "primary": r"Price target: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Recommendation: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Disclosures"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    }
}



In [6]:
def check_validity(paragraph):
    # Parse the paragraph
    paragraph = re.sub(r"\s+", " ", paragraph).strip()
    doc = nlp(paragraph)
        
    for sent in doc.sents:
        has_verb = False
        has_subject = False
        
        for token in sent:
            # Check for a verb
            if token.pos_ in {"VERB", "AUX"}:
                has_verb = True
            # Check for a subject
            if token.dep_ in {"nsubj", "nsubjpass"}:
                has_subject = True
        
        # If both a verb and a subject are found, the sentence is valid
        if has_verb and has_subject:
            return True
        
        # At least one word with 5+ letters and all upper case
        if re.search(r"[A-Z]{5,}", paragraph):
            return True
    return False

def filter_valid_paragraphs(paragraphs):
    valid_paragraphs = []

    for paragraph in paragraphs:
        if check_validity(paragraph):  # Validate each paragraph
            valid_paragraphs.append(paragraph)
    return valid_paragraphs

def extract_text_with_format(page, provider):
    #print(f"Provider: {provider}")

    provider_patterns = patterns[provider]

    # Access patterns
    price_patterns = provider_patterns["price_patterns"]
    rating_patterns = provider_patterns["rating_patterns"]
    ending_patterns = provider_patterns["ending_patterns"]
    font_patterns = provider_patterns["font_patterns"]

    # Extract words with font and size details
    words = page.extract_words(extra_attrs=["fontname", "size"])

    # Round text sizes to 3 decimal places, moved it to lower part, hope this is correct
    """for word in words:
        if "size" in word and word["size"] is not None:
            word["size"] = round(word["size"], 2)"""

    # Sort words by vertical and horizontal position
    words.sort(key=lambda w: (w["top"], w["x0"]))

    paragraphs = []
    current_paragraph = []
    current_top = None

    rating = None
    price = None

    # Lookahead buffer for multi-word patterns
    lookahead_buffer = []

    for word in words:

        if "size" in word and word["size"] is not None:
            word["size"] = round(word["size"], 2)

        # Build lookahead buffer
        lookahead_buffer.append(word["text"])
        if len(lookahead_buffer) > 10:
            lookahead_buffer.pop(0)
        buffer_text = " ".join(lookahead_buffer)

        #print(buffer_text)

        # Extract rating
        if not rating:
            for pattern_key in ["primary", "secondary","tertiary"]:
                pattern = rating_patterns.get(pattern_key)
                if pattern:
                    rating_match = re.search(pattern, buffer_text)
                    if rating_match:
                        rating = rating_match.group(1)
                        break

        # Extract price
        if not price:
            for pattern_key in ["primary", "secondary"]:
                pattern = price_patterns.get(pattern_key)
                if pattern:
                    price_match = re.search(pattern, buffer_text)
                    if price_match:
                        price = price_match.group(1)
                        break

        # Check for ending pattern
        # Check if buffer matches any ending pattern
        for ending_pattern in ending_patterns:
            if re.search(ending_pattern, buffer_text):
                return filter_valid_paragraphs(paragraphs), True, rating, price

        # Match word against font patterns
        is_font_matched = False
        for font_pattern in font_patterns:
            font_type = font_pattern["font_type"]
            font_size = font_pattern["font_size"]
            if re.match(font_type, word["fontname"]) and word["size"] == font_size:
                is_font_matched = True
                #break

        if not is_font_matched:
            continue

        # Group words into paragraphs
        if current_top is None or abs(word["top"] - current_top) < 13:  # Adjust threshold as needed
            current_paragraph.append(word["text"])
        else:
            # New paragraph starts
            paragraphs.append(" ".join(current_paragraph))
            current_paragraph = [word["text"]]

        # Update the current top position
        current_top = word["top"]

    # Add the last paragraph
    if current_paragraph:
        paragraphs.append(" ".join(current_paragraph))

    return filter_valid_paragraphs(paragraphs), False, rating, price


# NEW VERSION (16.11.2024)
def extract_metadata(filename, ticker_map):
    """
    Extract metadata (date, provider, ticker) from the filename using ticker_map.
    """
    # Extract the date (first 8 digits in the filename)
    date_match = re.match(r"(\d{8})", filename)
    if not date_match:
        return None, None, None, None, None
    date_str = date_match.group(1)
    date = datetime.strptime(date_str, "%Y%m%d")

    #print(f"fileName: {filename}")
    #print(f"date: {date}")

    # Look for the ticker in the filename
    for ticker in ticker_map.keys():
        ticker_pattern = f"_{ticker}_"  # Ensure ticker is surrounded by underscores
        if ticker_pattern in filename:
            # Extract the portion between date and ticker as the provider
            provider_section = filename.split(f"{date_str}_")[1].split(f"_{ticker}_")[0]
            provider = provider_section.replace('_', ' ')  # Replace underscores with spaces
            # Get company name and industry from the ticker_map
            company_name = ticker_map[ticker]['Company Name']
            industry = ticker_map[ticker]['Industry']
            return date, provider, ticker, company_name, industry

    # If no ticker is found, return None for ticker-related fields
    return date, None, None, None, None


In [8]:

# Define function to process all PDFs in the directory and store data in DataFrame
def extract_text_from_all_pdfs_to_dataframe(directory_path, provider_info, ticker_map):
    data = []
    id_counter = 1  # Initialize an ID counter
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):  # Process only PDF files
            file_path = os.path.join(directory_path, filename)

            
            # Extract metadata from filename
            date, provider, ticker, company_name, industry = extract_metadata(filename,ticker_map=ticker_map)

            all_paragraphs = []  
            first_rating = None
            first_price = None  
            stop_extraction = False

            with pdfplumber.open(file_path) as pdf:
                for page_number, page in enumerate(pdf.pages, start=1):
                    if stop_extraction:
                        break  # Exit the loop if stop_extraction is set
            
                    # Extract data from the current page
                    paragraphs, stop_extraction, rating, price = extract_text_with_format(page, provider=provider)
                    
                    # Append paragraphs from the current page
                    all_paragraphs.extend(paragraphs)

                    # Capture the first non-None rating and price
                    if rating is not None and first_rating is None:
                        first_rating = rating
                    if price is not None and first_price is None:
                        first_price = price
                        
            # Add extracted data to the list
            data.append({
                "document_id": id_counter,  # Unique ID
                "filename": filename,
                "date": date,
                "provider": provider,
                "ticker": ticker,
                "company_name": company_name,
                "industry": industry,
                "paragraphs": all_paragraphs,
                "target_price":first_price,
                "rating": first_rating.lower() if first_rating else None
                })
            id_counter += 1  # Increment the ID counter for the next row

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df

# Directory path
pdf_directory = "/Users/oskarroeske/Desktop/Analyst_Reports/Staging"  # Replace with your actual folder path

# Run the function and store results in a DataFrame
df_reports = extract_text_from_all_pdfs_to_dataframe(pdf_directory, provider_info, ticker_map)

# Display the resulting DataFrame to confirm
df_reports.head(15)

,document_id,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating
0,1,20150305_Needham_CRM_CRM-_Not_Even_FX_Could_Ta...,2015-03-05,Needham,CRM,Salesforce.com Inc.,Technology,"[BUY INVESTMENT HIGHLIGHTS:, Q4 was solid, hig...",80.00,buy
1,2,20171002_JP_Morgan_NEM_Denver_Gold_Show-_Nevad...,2017-10-02,JP Morgan,NEM,Newmont Corp.,Materials,[],None,overweight
2,3,20160510_Phillip_Securities_AAPL_Apple_Inc._Th...,2016-05-10,Phillip Securities,AAPL,Apple Inc.,Technology,[Apple Inc. Snapshot Apple Inc. (AAPL) recentl...,None,None
3,4,20170322_BTIG_NKTR_Nektar_Therapeutics.pdf,2017-03-22,BTIG,NKTR,Nektar Therapeutics,Healthcare,[Nektar announced positive results from the Ph...,22.00,buy
4,5,20180504_Needham_AMZN_Momentum_Remains_High_wi...,2018-05-04,Needham,AMZN,Amazon.com Inc.,Consumer Discretionary,"[BUY INVESTMENT HIGHLIGHTS:, Amazon posted str...","1,900.00",buy
5,6,20181114_BTIG_PRGO_Perrigo_Company_plc.pdf,2018-11-14,BTIG,PRGO,Perrigo,Healthcare,[(212) 527-3505 We have lowered our CY18 reven...,None,None
6,7,20150819_BTIG_PRGO_Perrigo_Company_plc.pdf,2015-08-19,BTIG,PRGO,Perrigo,Healthcare,[],223.00,buy
7,8,20151223_Needham_TSLA_SCTY-_SCTY_Analyst_Day_H...,2015-12-23,Needham,TSLA,Tesla Inc.,Automobile,[INVESTMENT HIGHLIGHTS: NA SCTY hosted a well-...,None,hold
8,9,20150615_Gilford_Securities_Inc_WMT_Report_rec...,2015-06-15,Gilford Securities Inc,WMT,Walmart Inc.,Consumer Staples,[],None,None


# Cleaning of paragraphs

In [9]:
import re

def clean_paragraph(paragraph):
    # Remove email addresses
    paragraph = re.sub(r"\S+@\S+", "", paragraph)
    
    # Remove phone numbers
    paragraph = re.sub(r"\b\d{1,3}[-.\s]?\d{3}[-.\s]?\d{4}\b", "", paragraph)
    
    # Remove URLs
    paragraph = re.sub(r"http\S+|www\S+", "", paragraph)
    
    # Remove special characters (keep alphanumerics, spaces, and common punctuation)
    paragraph = re.sub(r"[^\w\s,.!?]", "", paragraph)
    
    # Remove multiple spaces or newlines
    paragraph = re.sub(r"\s+", " ", paragraph).strip()
    
    # Remove boilerplate phrases
    boilerplate_phrases = ["Disclaimer", "Confidential", "For internal use only"]
    for phrase in boilerplate_phrases:
        paragraph = paragraph.replace(phrase, "")
    
    # Remove short sentences (fewer than 10 words)
    if len(paragraph.split()) < 6:
        return None
    
    return paragraph

# Apply cleaning function
df_reports['paragraphs'] = df_reports['paragraphs'].apply(
    lambda paragraphs: [clean_paragraph(p) for p in paragraphs if clean_paragraph(p)]
)

# Drop rows where the `paragraphs` column is empty after cleaning
df_reports = df_reports[df_reports['paragraphs'].str.len() > 0]

df_reports["rating"] = df_reports["rating"].astype(str).apply(lambda x: x.lower() if x != "nan" else None)

# Get Date and Price for Dates

In [10]:
df_reports.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 0 to 7
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   document_id   6 non-null      int64         
 1   filename      6 non-null      object        
 2   date          6 non-null      datetime64[ns]
 3   provider      6 non-null      object        
 4   ticker        6 non-null      object        
 5   company_name  6 non-null      object        
 6   industry      6 non-null      object        
 7   paragraphs    6 non-null      object        
 8   target_price  3 non-null      object        
 9   rating        6 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 528.0+ bytes


In [11]:
# Load and ensure 'Date' is in datetime format
df_performance_data = pd.read_csv("performance_data.csv")

# Convert 'Date' column to datetime and set it as index (with timezone awareness)
df_performance_data['Date'] = pd.to_datetime(df_performance_data['Date'], utc=True)
df_performance_data = df_performance_data.set_index('Date')

In [12]:
df_saved_reports = df_reports

In [13]:
def get_stock_prices(ticker, start_date, end_date=None):
    try:
        # Ensure start_date and end_date are in the same timezone (UTC)
        start_date = pd.to_datetime(start_date, utc=True)
        if end_date is not None:
            end_date = pd.to_datetime(end_date, utc=True)
        else:
            end_date = start_date  # If no end_date, use start_date for a single day

        # Filter the data for the ticker and date range
        filtered_data = df_performance_data.loc[
            (df_performance_data.index >= start_date) & 
            (df_performance_data.index <= end_date), ticker]

        if not filtered_data.empty:
            # Return the maximum price within the filtered date range
            max_price = filtered_data.max()
            min_price = filtered_data.min()
            return float(max_price), float(min_price)
        else:
            # If no data available in the range, find the next available date using asof()
            next_available_data = df_performance_data[ticker].asof(start_date)
            if next_available_data is not None:
                return float(next_available_data), float(next_available_data)
            else:
                return float('nan'), float('nan')  # Return NaN for missing data

    except KeyError:
        return float('nan'), float('nan')  # Return NaN for missing data
    except Exception as e:
        return float('nan'), float('nan')  # Return NaN for missing data

def calculate_prices(row):
    short_name = row['ticker']
    base_date = pd.to_datetime(row['date'], utc=True)

    # Calculate prices
    row['start price'] = get_stock_prices(short_name, base_date)[0]
    row['one day after'] = get_stock_prices(short_name, base_date + pd.DateOffset(days=1))[0]
    row['max price after 3 months'], row["min price after 3 months"] = get_stock_prices(
        short_name, base_date, base_date + pd.DateOffset(months=3)
    )
    row['max price after 6 months'], row['min price after 6 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=3), base_date + pd.DateOffset(months=6)
    )
    row['max price after 9 months'], row['min price after 9 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=6), base_date + pd.DateOffset(months=9)
    )
    row['max price after 12 months'], row['min price after 12 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=9), base_date + pd.DateOffset(months=12)
    )
    return row

# Apply the function to each row
df_saved_reports = df_saved_reports.apply(calculate_prices, axis=1)


In [14]:
df_saved_reports.head(10)

,document_id,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating,start price,one day after,max price after 3 months,min price after 3 months,max price after 6 months,min price after 6 months,max price after 9 months,min price after 9 months,max price after 12 months,min price after 12 months
0,1,20150305_Needham_CRM_CRM-_Not_Even_FX_Could_Ta...,2015-03-05,Needham,CRM,Salesforce.com Inc.,Technology,"[Q4 was solid, highlighted by 32 billings grow...",80.00,buy,65.300110,64.274658,74.678482,63.398548,75.375381,64.881958,81.776962,68.077774,80.721649,53.811111
2,3,20160510_Phillip_Securities_AAPL_Apple_Inc._Th...,2016-05-10,Phillip Securities,AAPL,Apple Inc.,Technology,[Apple Inc. Snapshot Apple Inc. AAPL recently ...,None,none,21.402905,21.194420,25.063862,20.697264,27.238321,23.755497,30.791845,24.474806,35.807560,30.722088
3,4,20170322_BTIG_NKTR_Nektar_Therapeutics.pdf,2017-03-22,BTIG,NKTR,Nektar Therapeutics,Healthcare,[Nektar announced positive results from the Ph...,22.00,buy,22.889999,22.530001,24.200001,17.540001,23.230000,17.790001,57.990002,22.260000,108.440002,57.400002
4,5,20180504_Needham_AMZN_Momentum_Remains_High_wi...,2018-05-04,Needham,AMZN,Amazon.com Inc.,Consumer Discretionary,[Amazon posted strong Q1 results with revenue ...,"1,900.00",buy,79.047501,79.047501,93.180496,78.718498,101.975502,76.521004,88.617996,67.197998,98.123001,79.411003
5,6,20181114_BTIG_PRGO_Perrigo_Company_plc.pdf,2018-11-14,BTIG,PRGO,Perrigo,Healthcare,[212 5273505 We have lowered our CY18 revenue ...,None,none,54.523006,54.565964,55.247051,31.454027,45.529312,40.234013,47.905579,36.526798,49.510155,39.491028
7,8,20151223_Needham_TSLA_SCTY-_SCTY_Analyst_Day_H...,2015-12-23,Needham,TSLA,Tesla Inc.,Automobile,[INVESTMENT HIGHLIGHTS NA SCTY hosted a wellat...,None,hold,15.313333,15.371333,16.000668,9.578000,17.694668,13.093333,15.652667,12.876667,14.246667,12.096667


In [37]:
df_saved_reports["target_price"] = (
    df_saved_reports["target_price"]
    #Remove comma and transform to float for later operations
    .replace(",", "", regex=True) 
    .astype(float)                 
)

# Calculate performance of Target Prices

In [38]:
#If the target_price is below the current price, we need to check if it was below the target_price
def classifcy_performance(df):
    if df["target_price"] > df["start price"]:
        df["tp reached after 3 months"] = df["target_price"] <= df["max price after 3 months"]
        df["tp reached after 6 months"] = df["target_price"] <= df["max price after 6 months"]
        df["tp reached after 9 months"] = df["target_price"] <= df["max price after 9 months"]
        df["tp reached after 12 months"] = df["target_price"] <= df["max price after 12 months"]
    else:
        df["tp reached after 3 months"] = df["target_price"] >= df["min price after 3 months"]
        df["tp reached after 6 months"] = df["target_price"] >= df["min price after 6 months"]
        df["tp reached after 9 months"] = df["target_price"] >= df["min price after 9 months"]
        df["tp reached after 12 months"] = df["target_price"] <= df["min price after 12 months"]

    return df

df_saved_reports = df_saved_reports.apply(classifcy_performance, axis=1)

# Clean/Filter Paragraphs

In [39]:
df_saved_reports.value_counts("rating")

rating
buy     2
none    2
hold    1
Name: count, dtype: int64

In [40]:
#df_saved_reports.to_csv("/Users/oskarroeske/Masterthesis/full_analysis/preprocessed_reports.csv")

In [41]:
# Explode the cleaned list into separate rows
df_exploded = df_saved_reports.explode('paragraphs').reset_index(drop=True)

In [43]:
df_exploded = df_exploded[["document_id","paragraphs"]]

In [50]:
df_exploded['paragraph_id'] = df_exploded.groupby('document_id').cumcount() + 1

/var/folders/2h/tyzkflxs465698k6z8l3ktj00000gn/T/ipykernel_99240/3036036435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exploded['paragraph_id'] = df_exploded.groupby('document_id').cumcount() + 1


In [51]:
df_exploded

,document_id,paragraphs,paragraph_id
0,1,"Q4 was solid, highlighted by 32 billings growt...",1
1,1,While F16 rev guide was only tweaked up mildly...,2
2,1,Other highlights included a solid top line per...,3
3,1,Q4 revenue was up 26 yy to 1.44B and in line w...,4
4,1,"Management was upbeat on the call, especially ...",5
...,...,...,...
78,7,We appreciate mgmt for offering more simplifie...,5
79,7,We were impressed by the demo of the Silevo pi...,6
80,7,New subsidy programs or new states where Solar...,7
81,7,Leadership position could become a greater bar...,8


In [55]:
#df_exploded.to_csv("/Users/oskarroeske/Masterthesis/full_analysis/preprocessed_paragraphs.csv")

# Turn paragraphs into sentences

In [62]:
df_exploded

,document_id,paragraphs,paragraph_id
0,1,"Q4 was solid, highlighted by 32 billings growt...",1
1,1,While F16 rev guide was only tweaked up mildly...,2
2,1,Other highlights included a solid top line per...,3
3,1,Q4 revenue was up 26 yy to 1.44B and in line w...,4
4,1,"Management was upbeat on the call, especially ...",5
...,...,...,...
78,7,We appreciate mgmt for offering more simplifie...,5
79,7,We were impressed by the demo of the Silevo pi...,6
80,7,New subsidy programs or new states where Solar...,7
81,7,Leadership position could become a greater bar...,8


In [66]:
import spacy
import pandas as pd

# Load spaCy language model
nlp = spacy.load("en_core_web_sm")

# Prepare an empty list to store the results
result = []

# Function to split text into sentences using spaCy
def split_into_sentences(text):
    doc = nlp(text)
    return [sent.text for sent in doc.sents]

# Split each paragraph into sentences
for _, row in df_exploded.iterrows():
    doc_id = row['document_id']
    paragraph_id = row['paragraph_id']
    text = row['paragraphs']
    sentences = split_into_sentences(text)
    for sentence_id, sentence in enumerate(sentences, start=1):
        # Check if the sentence has more than 3 words
        if len(sentence.split()) >= 3:
            result.append({
                'document_id': doc_id,
                'paragraph_id': paragraph_id,
                'sentence_id': sentence_id,
                'sentence': sentence
            })

# Create a new DataFrame with the results
df_sentences = pd.DataFrame(result)

In [67]:
df_sentences

,document_id,paragraph_id,sentence_id,sentence
0,1,1,1,"Q4 was solid, highlighted by 32 billings growt..."
1,1,1,2,One of the highlights on the call was the earl...
2,1,1,3,Another standout on the call was the number of...
3,1,2,1,While F16 rev guide was only tweaked up mildly...
4,1,2,2,"Encouragingly, management appeared committed t..."
...,...,...,...,...
255,7,6,1,We were impressed by the demo of the Silevo pi...
256,7,6,2,We believe these technologies will help SCTY r...
257,7,7,1,New subsidy programs or new states where Solar...
258,7,8,1,Leadership position could become a greater bar...


In [68]:
df_sentences.to_csv("/Users/oskarroeske/Masterthesis/full_analysis/preprocessed_sentences.csv")

In [56]:
#df_reports.to_csv("paragraphs.csv")